# Get data for all games on https://www.metacritic.com/browse/games/release-date/new-releases/all/date 

In [77]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

In [78]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

# Get table rows from multiple pages 

In [79]:
trs=[]
for i in range(1):
    url=f"https://www.metacritic.com/browse/games/release-date/new-releases/pc/date?page={i}"
    page=requests.get(url,headers=headers).content
    soup=bs(page,"html.parser")
    mylist=["one","two","three","four"]
    trs1=[]
    for x in mylist:
        tr=soup.select(f"div[class='browse_list_wrapper {x} browse-list-large'] table[class='clamp-list'] tr td[class='clamp-summary-wrap'] a[class='title']")
        trs1=trs1+tr
    trs=trs+trs1

# get data from each table row

In [80]:
all_data=pd.DataFrame()
for tr in trs:
    title=tr.text
    href=tr["href"]
    url2="https://www.metacritic.com/"+href
    soup=bs(requests.get(url2,headers=headers).content,"html.parser")
    date=soup.select_one("div[class='content_head product_content_head game_content_head'] ul[class='summary_details'] li[class='summary_detail release_data'] span[class='data']")
    infos=soup.select("div[class='module product_data product_data_summary'] ul[class='summary_details'] span[class='label']")
    datas=soup.select("div[class='module product_data product_data_summary'] ul[class='summary_details'] span[class='data']")
    metascore=soup.select_one("div[class='section product_scores'] span[itemprop='ratingValue']")
        
        
    summary={}
    for info,data in zip(infos,datas):

        summary[info.text]=data.text.replace("\n",""),
        summary["date"]=date.text,
        summary["metascore"]=[metascore.text if metascore is not None else "No Score"]
    df=pd.DataFrame(summary)
    df.insert(0,"Title",tr.text)
    all_data=pd.concat([all_data,df],ignore_index=True)

# Clean Data

In [81]:
df=all_data.loc[:,all_data.columns.isin(["Cheats:","# of players:"])==False]
df=df.rename(columns={"Developer:":"Developer","date":"Date","metascore":"Metascore","Genre(s):":"Genre","Summary:":"Summary"})
df

,Title,Developer,Date,Metascore,Genre(s):,Summary,Available On:,Rating:
0,TRATRITLE,INAZUMA SOFT,"Jul 16, 2022",No Score,Adventure,NaN,NaN,NaN
1,Death Leak,Jay Bee,"Jul 16, 2022",No Score,Action,NaN,NaN,NaN
2,Shape Puzzle,Devment Studio,"Jul 15, 2022",No Score,Puzzle,"Combine, rotate and move different shapes to s...",NaN,NaN
3,Knight Princess Eris,WILD FLOWER,"Jul 15, 2022",No Score,Adventure,"“Knight Princess Eris"" is an action game with ...",NaN,NaN
4,Harem of Nurses,BigGuy Games,"Jul 15, 2022",No Score,Action,"By chance, I acquired a powerful hypnosis devi...",NaN,NaN
...,...,...,...,...,...,...,...,...
95,Loopmancer,eBrain Studio,"Jul 13, 2022",77,Action,"Year 2046 / Dragon City. Xiang Zixu, a brillia...",NaN,NaN
96,Elena's Journal - Unfinished Expedition,SQRT3,"Jul 13, 2022",No Score,Strategy,Help Elena discover the City of the Amazons in...,NaN,NaN
97,Man of the World,redduckstudio,"Jul 13, 2022",No Score,Role-Playing,"Role-playing game in an open world. Battles, m...",NaN,NaN
98,Kung Fu Er San,CMC Games,"Jul 13, 2022",No Score,Adventure,Awesome 2D retro pixel fighting game inspired ...,NaN,NaN
